# Import der Packages und der Datengrundlage

In [1]:
import numpy as np
import cv2
import tensorflow as tf
from keras.models import load_model
best_model = load_model("Deployment/Finetune_VGG_5_SGD_32,0.005_DA.h5") #Einlesen eines Modells zur Klassifikation
face_haar_cascade = cv2.CascadeClassifier("Deployment/haarcascade_frontalface_default.xml") #Face Haar Cascade Algorithmus zur Gesichtserkennung

In [2]:
#Echtzeit Emotionserkennung
cap = cv2.VideoCapture(0)
cap.set(3,640) 
cap.set(4,480) 
while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=5,minSize=(20, 20))
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),2) 
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi_color = img[y:y+h, x:x+w]
        roi_color = cv2.resize(roi_color, (48, 48))
        img_pixels = tf.keras.utils.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        
        predictions = best_model.predict(img_pixels)
        max_index = np.argmax(predictions[0])

        emotion_label = ("Wut", "Ekel", "Angst", "Freude", "Trauer", "Ueberraschung", "Neutral")
        predicted_emotion = emotion_label[max_index]

        cv2.putText(img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        
    cv2.imshow('video',img) 
    k = cv2.waitKey(30) & 0xff

    if k == 27: 
        break
cap.release()
cv2.destroyAllWindows()